In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
path = '/content/gdrive/My Drive/Flipr 6//'

In [3]:
# Import Useful Libraries :
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
# Import Datasets :
train = pd.read_excel(path+'Train-dataset.xlsx')
test1 = pd.read_excel(path+'Test-dataset.xlsx', sheet_name = "Test_data" )

## **Variable Description :**
![alt text](http://drive.google.com/uc?id=1fR28CvHs5L7PlxDF0pMwE6aUDrNWxfve)

### Data type , Unique count &  Missing Count for all columns in Train & Test dataset :

In [5]:
train_unique = [] ;train_missing = []
test1_unique = []; test1_missing = [] 
data_type =[]

for col in test1 :

    #data type
    data_type.append(str(train[col].dtype))

    
    # unique values
    train_unique.append(train[col].nunique()) 
    test1_unique.append(test1[col].nunique())
    
    # missing values
    train_missing.append( train[col].isna().sum())
    test1_missing.append( test1[col].isna().sum())

df = pd.DataFrame({"Column":test1.columns," Data_Type":data_type,"Train_Unique":train_unique,"Train_Missing":train_missing,"Test1_Unique":test1_unique,"Test1_Missing":test1_missing})

df

,Column,Data_Type,Train_Unique,Train_Missing,Test1_Unique,Test1_Missing
0,City,object,772,0,497,0
1,State,object,33,0,29,0
2,Type,object,37,0,32,2
3,Population [2011],float64,730,48,486,6
4,Popuation [2001],float64,292,492,0,501
5,Sex Ratio,float64,160,10,145,5
6,Median Age,float64,10,18,10,13
7,Avg Temp,float64,26,17,27,14
8,SWM,object,3,4,3,9
9,Toilets Avl,float64,107,26,62,22


In [6]:
print(train["SWM"].unique(),test1["SWM"].unique())

['MEDIUM' 'HIGH' 'LOW' nan] ['MEDIUM' 'HIGH' 'LOW' nan]


### Observations :
- SWM is a categorical variable.
- State and Foreign Visitors are realted .

In [7]:
# Making "State" , "City" & "SWM" columns case-insensitive :

train["State"] = train["State"].str.upper()
test1["State"] = test1["State"].str.upper()

train["City"] = train["City"].str.upper()
test1["City"] = test1["City"].str.upper()

train["SWM"] = train["SWM"].str.upper()
test1["SWM"] = test1["SWM"].str.upper()

In [8]:
# Sorting dataset by State and City :
train = train.sort_values(by=['State', 'City'], ascending=[True, True]).reset_index(drop=True)
test1 = test1.sort_values(by=['State', 'City'], ascending=[True, True]).reset_index(drop=True)

In [9]:
# All unique cominations of "State" and "Foreign Visitors" :
state_visitor = pd.concat([train[["State","Foreign Visitors"]],test1[["State","Foreign Visitors"]]])
state_visitor = state_visitor.dropna()
state_visitor = state_visitor.drop_duplicates(keep='first')

In [10]:
state_visitor
#state_visitor.shape
#len( (set(train["State"].unique()))  .union (set(test1["State"].unique())))

,State,Foreign Visitors
0,ANDAMAN AND NICOBAR ISLANDS,14674.0
1,ANDHRA PRADESH,237854.0
79,ASSAM,24720.0
100,BIHAR,923737.0
151,CHANDIGARH,29538.0
152,CHHATTISGARH,6394.0
164,DELHI,2379169.0
170,GOA,541480.0
173,GUJARAT,284973.0
216,HARYANA,303118.0


In [11]:
# "Foreign Visitors" column with  missing values filled according to state:

train = pd.merge(train, state_visitor.drop(columns=[]), how='left', on=['State'])
train = train.drop(columns=['Foreign Visitors_x'])
train = train.rename(columns={'Foreign Visitors_y': 'Foreign Visitors'})
#train

test1 = pd.merge(test1, state_visitor.drop(columns=[]), how='left', on=['State'])
test1 = test1.drop(columns=['Foreign Visitors_x'])
test1 = test1.rename(columns={'Foreign Visitors_y': 'Foreign Visitors'})
#test1


In [12]:
# Replacing SWM values with numerical values :

SWM_label = {
             "HIGH":1,
             "MEDIUM":0.5,
             "LOW":0
            }


train["SWM"]=[SWM_label[i] if i in SWM_label.keys() else i for i in train["SWM"] ]
test1["SWM"]=[SWM_label[i] if i in SWM_label.keys() else i for i in test1["SWM"] ]

In [13]:
# Replacing missing values with  mean of that column 
cols = ['Population [2011]', 'Sex Ratio', 'Median Age', 'Avg Temp', 'SWM','Toilets Avl', 'Water Purity', 'H Index', 'Female Population','# of hospitals']
for col in cols:
    train[col] = train.groupby("State").transform(lambda x: x.fillna(x.mean()))[col]
    test1[col] = test1.groupby("State").transform(lambda x: x.fillna(x.mean()))[col]

In [14]:
train.corr()

,Population [2011],Popuation [2001],Sex Ratio,Median Age,Avg Temp,SWM,Toilets Avl,Water Purity,H Index,Female Population,# of hospitals,Covid Cases,Foreign Visitors
Population [2011],1.000000,0.966254,-0.081281,-0.033557,0.025932,0.000399,0.475003,-0.010201,0.022735,0.980704,0.399090,0.729698,0.100947
Popuation [2001],0.966254,1.000000,-0.110252,-0.033197,0.017470,0.041770,0.484772,0.033492,0.064624,0.958287,0.321359,0.679870,0.136163
Sex Ratio,-0.081281,-0.110252,1.000000,0.152479,0.014667,0.026978,-0.058366,0.070112,0.032212,-0.059637,-0.081068,-0.029691,-0.196343
Median Age,-0.033557,-0.033197,0.152479,1.000000,-0.046075,0.042822,-0.041495,0.001905,0.022436,-0.029983,-0.052769,0.008718,-0.023574
Avg Temp,0.025932,0.017470,0.014667,-0.046075,1.000000,-0.028557,0.035157,0.004368,0.074803,0.023043,0.059053,0.019439,0.154869
SWM,0.000399,0.041770,0.026978,0.042822,-0.028557,1.000000,0.005639,0.029265,-0.024471,-0.009898,-0.042076,0.013438,-0.065488
Toilets Avl,0.475003,0.484772,-0.058366,-0.041495,0.035157,0.005639,1.000000,-0.061839,-0.004199,0.445364,0.756965,0.397329,0.061810
Water Purity,-0.010201,0.033492,0.070112,0.001905,0.004368,0.029265,-0.061839,1.000000,-0.004624,-0.006228,-0.015531,0.029270,-0.034034
H Index,0.022735,0.064624,0.032212,0.022436,0.074803,-0.024471,-0.004199,-0.004624,1.000000,0.027346,-0.004934,-0.009559,-0.014387
Female Population,0.980704,0.958287,-0.059637,-0.029983,0.023043,-0.009898,0.445364,-0.006228,0.027346,1.000000,0.378616,0.731727,0.096576


In [15]:
def features(df):
    aggs = {}

    for  col  in ['Population [2011]', 'Sex Ratio', 'Median Age', 'Avg Temp', 'SWM','Toilets Avl', 'Water Purity', 'H Index', 'Female Population','# of hospitals', 'Foreign Visitors', 'Covid Cases']:
        aggs[col] = ['max','min','mean']
    agg_df = df.groupby('State').agg(aggs)

    agg_df = agg_df.reset_index()

    return agg_df

In [16]:
df = features(train)

df.columns  =  ['State'] +  [i[0] + '__' + i[1]  for i in df.columns[1:]]

In [17]:
train = pd.merge(train,df, left_on='State', right_on='State', how='left')
train = train [[
                'State','City','Type',
                'Population [2011]','Population [2011]__mean','Population [2011]__max','Population [2011]__min',
                'Sex Ratio' , 'Sex Ratio__mean','Sex Ratio__max','Sex Ratio__min', 
                'Median Age', 'Median Age__mean','Median Age__max','Median Age__min', 
                'Avg Temp','Avg Temp__mean', 'Avg Temp__max', 'Avg Temp__min',
                'SWM' , 'SWM__mean','SWM__max','SWM__min',
                'Toilets Avl', 'Toilets Avl__mean','Toilets Avl__max', 'Toilets Avl__min',
                'Water Purity', 'Water Purity__mean' ,'Water Purity__max', 'Water Purity__min',
                'H Index', 'H Index__mean','H Index__max', 'H Index__min',
                'Female Population','Female Population__mean','Female Population__max', 'Female Population__min',
                '# of hospitals','# of hospitals__mean','# of hospitals__max', '# of hospitals__min',
                'Foreign Visitors','Foreign Visitors__mean','Foreign Visitors__max','Foreign Visitors__min', 
                'Covid Cases','Covid Cases__mean','Covid Cases__max','Covid Cases__min'
             ]]

In [18]:
_dict={}
_dict["feature"]= list(train.columns[3:])

methods = ["pearson","spearman","kendall"]

for i in methods :
     _dict[i] = list(train.corr(method=i)['Covid Cases'])

corr = pd.DataFrame(_dict)

corr

,feature,pearson,spearman,kendall
0,Population [2011],0.729698,0.575424,0.394088
1,Population [2011]__mean,0.155684,-0.037315,-0.026225
2,Population [2011]__max,0.192937,0.041942,0.028018
3,Population [2011]__min,0.029702,-0.012356,-0.007835
4,Sex Ratio,-0.029691,0.129959,0.096972
5,Sex Ratio__mean,-0.023989,0.139802,0.102864
6,Sex Ratio__max,-0.030169,0.125669,0.093069
7,Sex Ratio__min,-0.016417,0.147050,0.107253
8,Median Age,0.008718,0.285136,0.240890
9,Median Age__mean,-0.023719,0.157128,0.108922


In [19]:
# important_features = ["Population [2011]","SWM","Toilets Avl","Water Purity","# of hospitals","Foreign Visitors"]

In [20]:
from catboost import Pool, CatBoostRegressor
features = [
            #'Population [2011]', 
            'Population [2011]__mean',
            'Population [2011]__max', 
            'Population [2011]__min', 
            'Sex Ratio',
            'Sex Ratio__mean', 
            #'Sex Ratio__max', 
            'Sex Ratio__min',
            'Median Age',
            'Median Age__mean', 
            #'Median Age__max', 
            #'Median Age__min',
            'Avg Temp',
            #'Avg Temp__mean', 
            #'Avg Temp__max', 
            #'Avg Temp__min', 
            'SWM', 
            #'SWM__mean',
            #'SWM__max', 
            #'SWM__min',
            'Toilets Avl', 
            #'Toilets Avl__mean',
            #'Toilets Avl__max',
            #'Toilets Avl__min', 
            'Water Purity',
            #'Water Purity__mean', 
            #'Water Purity__max',
            #'Water Purity__min',
            'H Index', 
            #'H Index__mean', 
            'H Index__max', 
            'H Index__min',
            #'Female Population', 
            #'Female Population__mean',
            #'Female Population__max', 
            #'Female Population__min', 
            '# of hospitals',
            #'# of hospitals__mean', 
            #'# of hospitals__max', 
            #'# of hospitals__min',
            'Foreign Visitors', 
            #'Foreign Visitors__mean', 
            #'Foreign Visitors__max',
            #'Foreign Visitors__min', 
            'Covid Cases__mean',
            'Covid Cases__max', 
            'Covid Cases__min'
         ]
            
features = ['# of hospitals',
#  '# of hospitals__max',
#  '# of hospitals__mean',
#  '# of hospitals__min',
 'Avg Temp',
#  'Avg Temp__max',
#  'Avg Temp__mean',
#  'Avg Temp__min',
 'Covid Cases__max',
 'Covid Cases__mean',
 'Covid Cases__min',
#  'Female Population',    # ?
#  'Foreign Visitors',
 'Foreign Visitors__max',
#  'Foreign Visitors__mean',
#  'Foreign Visitors__min',
 'H Index',
 'H Index__max',
#  'H Index__mean',
 'H Index__min',
 'Median Age',
#  'Median Age__max',
 'Median Age__mean',
#  'Median Age__min',
#  'Popuation [2001]',
 'Population [2011]',
 'Population [2011]__max',
 'Population [2011]__mean',
 'Population [2011]__min',
 'SWM',
#  'SWM_label',
#  'SWM_label__max',
#  'SWM_label__mean',
#  'SWM_label__min',
 'Sex Ratio',
#  'Sex Ratio__max',
 'Sex Ratio__mean',
 'Sex Ratio__min',
 'Toilets Avl',
#  'Toilets Avl__max',
#  'Toilets Avl__mean',
#  'Toilets Avl__min',
#  'Type',
 'Water Purity',
#  'Water Purity__max',
#  'Water Purity__mean',
#  'Water Purity__min'
]


features = [i[0] for i in [('Population [2011]', 28.697789374493304),
                            ('Median Age', 10.831209218393639),
                            ('Sex Ratio', 10.342068479139515),
                            ('Toilets Avl', 6.357879781121504),
                            ('H Index', 6.279388057622039),
                            ('# of hospitals', 5.470589711238431),
                            ('Covid Cases__mean', 4.958544149313693),
                            ('Water Purity', 3.2682574262132937),
                            ('Sex Ratio__min', 3.248341107087464),
                            ('SWM', 3.122230536264299),
                            ('Avg Temp', 3.0037385688599585)]]




import time
from sklearn.utils import shuffle
import numpy as np

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


from sklearn.metrics import mean_absolute_error#, mean_absolute_percentage_error

average = [[],[]]; count = 20
for  _  in range(count):
    seed = int(str(time.time()).split('.')[-1][-2:])
    print(seed)

    # Train test split
    df = train.copy()
    df  =  shuffle(df, random_state =seed)

    train__, test__  =  df.iloc[:int(0.8*len(df))], df.iloc[int(0.8*len(df)):]



    model = CatBoostRegressor(loss_function='MAE', verbose=False, random_seed = seed)
    model.fit(Pool(train__[features], train__['Covid Cases']))
    preds = model.predict(test__[features])



    actual = test__['Covid Cases']
    print(mean_absolute_error(actual, preds), mean_absolute_percentage_error(actual, preds))
    average[0] += [mean_absolute_error(actual, preds)]
    average[1] += [mean_absolute_percentage_error(actual, preds)]

print(np.mean(np.array(average[0])), np.mean(np.array(average[1])))    # MAE, MAPE


24
1899.7705510290991 28.394362238713196
97
1740.9861703525478 35.223152355663316
28
2438.081674711235 43.089216012223666
13
1898.4052574558598 30.048621150496622
48
3445.0178707117593 36.00243307671623
69
2359.573599075252 22.65177408843507
94
2431.4234353105303 24.77353175366293
78
3251.617908891887 58.93311052897959
54
1408.496284717447 40.574109621655566
59
1585.2064681021338 29.686695016856547
22
2810.093011497919 28.511780452317627
42
1351.4184307058958 32.50158525590257
42
1351.4184307058958 32.50158525590257
58
1172.6342664680756 27.387786000576824
13
1898.4052574558598 30.048621150496622
78
3251.617908891887 58.93311052897959
65
2286.602193601194 39.22251577485004
45
2523.9136897293 32.0597251816343
66
2312.6140815528447 22.886136873266327
42
1351.4184307058958 32.50158525590257
2138.435746083626 34.296571878661595


In [21]:
# 3096.2111247242033 31.25604280299113     # FeatureSet1, MAPE
# 3215.4068345899373 33.99186414290216     # FeatureSet2, MAPE
# 3132.9251051299702 32.98034043279099     # FeatureSet3, MAPE

# 2548.5117967862625 37.43537536319028     # FeatureSet1, MAE
# 2236.33188939539 30.549138734999406      # FeatureSet2, MAE
# 2203.0778426546995 31.529664949873116    # FeatureSet3, MAE


In [22]:
actual, preds = list(actual), list(preds)
# [actual[i]/preds[i] for i in range(len(actual))]

print(len(actual))
for  ratio  in [0.05, 0.1, 0.15, 0.2]:
    print(ratio, 100*len([1   for i in range(len(actual))   if 1-ratio<actual[i]/preds[i] < 1+ratio])/len(actual))


158
0.05 51.89873417721519
0.1 66.45569620253164
0.15 70.88607594936708
0.2 72.15189873417721


In [23]:
list(zip(actual, preds))

[(1277, 2007.3340480693391),
 (10218, 9665.9620499781),
 (2193, 2133.8962235956),
 (2006, 1832.7667163563306),
 (2628, 2628.6465070298464),
 (4764, 6259.240348997111),
 (8911, 8562.099272777312),
 (2347, 2414.6754770302555),
 (2165, 2223.2054081639494),
 (2172, 2404.719851281316),
 (2774, 8166.620408609941),
 (1285, 12439.377567233298),
 (9430, 6652.032611060602),
 (8951, 7950.662184370423),
 (14249, 6900.023455696779),
 (2808, 2920.3400775731716),
 (2797, 3595.923406192296),
 (10933, 10386.646832097093),
 (3695, 2775.0680725372526),
 (9361, 9015.322081066097),
 (2506, 2396.191950804748),
 (2534, 2535.2479728401117),
 (20839, 10954.213053995598),
 (2175, 2167.918858246671),
 (2306, 2321.8781459828833),
 (1958, 6498.765727437022),
 (2523, 2508.245174822524),
 (2464, 2624.2994344780627),
 (2387, 2378.948693942231),
 (3834, 13594.28430592113),
 (15003, 16485.004042352433),
 (2123, 2152.140364202295),
 (2674, 2701.606467439597),
 (9613, 5651.725489448107),
 (2382, 2299.956327357172),
 (253

In [30]:
list(zip(features, model.feature_importances_))

[('# of hospitals', 7.938801597534141),
 ('Avg Temp', 1.2311702134790523),
 ('Covid Cases__max', 0.9126893437446663),
 ('Covid Cases__mean', 3.2079527621862307),
 ('Covid Cases__min', 3.9318972672811467),
 ('Foreign Visitors', 1.9481899275872776),
 ('H Index', 3.3476218619680274),
 ('H Index__max', 0.6887154202617629),
 ('H Index__min', 1.0443905552781019),
 ('Median Age', 7.8037785106224735),
 ('Median Age__mean', 0.7612936689062385),
 ('Population [2011]', 39.43802595484376),
 ('Population [2011]__max', 1.6999861543051573),
 ('Population [2011]__mean', 0.4845575732110203),
 ('Population [2011]__min', 2.2561818434086467),
 ('SWM', 2.6602799692872514),
 ('Sex Ratio', 5.0893460919781734),
 ('Sex Ratio__mean', 0.8332250394281938),
 ('Sex Ratio__min', 1.8222654479829137),
 ('Toilets Avl', 9.347117220216823),
 ('Water Purity', 3.5525135764889195)]

In [25]:
from sklearn.utils import shuffle
train  =  shuffle(train)

In [26]:
!pip install pycaret==2.0
from pycaret.regression import *

In [27]:
train_, test_  =  train.iloc[:int(0.8*len(train))], train.iloc[int(0.8*len(train)):]
train.shape, train_.shape, test_.shape

((787, 51), (629, 51), (158, 51))

In [28]:
reg  =  setup(train_[features], target = 'Covid Cases')

SystemExit: ignored

In [ ]:
best_model = compare_models(fold=5)

In [ ]:
df.shape

In [ ]:
features

# Final Prediction on the Test Data with complete Training Data.



In [29]:
train = pd.read_excel(path+'Train-dataset.xlsx')
test1 = pd.read_excel(path+'Test-dataset.xlsx', sheet_name = "Test_data" )

train["State"] = train["State"].str.upper()
test1["State"] = test1["State"].str.upper()
# train["City"] = train["City"].str.upper()
# test1["City"] = test1["City"].str.upper()
train["SWM"] = train["SWM"].str.upper()
test1["SWM"] = test1["SWM"].str.upper()


# # Sorting dataset by State and City :
# train = train.sort_values(by=['State', 'City'], ascending=[True, True]).reset_index(drop=True)
# test1 = test1.sort_values(by=['State', 'City'], ascending=[True, True]).reset_index(drop=True)



# All unique cominations of "State" and "Foreign Visitors" :
state_visitor = pd.concat([train[["State","Foreign Visitors"]],test1[["State","Foreign Visitors"]]])
state_visitor = state_visitor.dropna()
state_visitor = state_visitor.drop_duplicates(keep='first')

# "Foreign Visitors" column with  missing values filled according to state:
train = pd.merge(train, state_visitor.drop(columns=[]), how='left', on=['State'])
train = train.drop(columns=['Foreign Visitors_x'])
train = train.rename(columns={'Foreign Visitors_y': 'Foreign Visitors'})
#train

test1 = pd.merge(test1, state_visitor.drop(columns=[]), how='left', on=['State'])
test1 = test1.drop(columns=['Foreign Visitors_x'])
test1 = test1.rename(columns={'Foreign Visitors_y': 'Foreign Visitors'})
#test1



# Replacing SWM values with numerical values :
SWM_label = {
             "HIGH":1,
             "MEDIUM":0.5,
             "LOW":0
            }

train["SWM"]=[SWM_label[i] if i in SWM_label.keys() else i for i in train["SWM"] ]
test1["SWM"]=[SWM_label[i] if i in SWM_label.keys() else i for i in test1["SWM"] ]




train_plus_test = pd.concat([train, test1])

# Replacing missing values with  mean of that column 
cols = ['Population [2011]', 'Sex Ratio', 'Median Age', 'Avg Temp', 'SWM','Toilets Avl', 'Water Purity', 'H Index', 'Female Population','# of hospitals']
for col in cols:
    train_plus_test[col] = train_plus_test.groupby("State").transform(lambda x: x.fillna(x.mean()))[col]



def create_features(df):
    aggs = {}

    for  col  in ['Population [2011]', 'Sex Ratio', 'Median Age', 'Avg Temp', 'SWM','Toilets Avl', 'Water Purity', 'H Index', 'Female Population','# of hospitals', 'Foreign Visitors', 'Covid Cases']:
        aggs[col] = ['max','min','mean']
    agg_df = df.groupby('State').agg(aggs)

    agg_df = agg_df.reset_index()

    return agg_df

df = create_features(train_plus_test)
df.columns  =  ['State'] +  [i[0] + '__' + i[1]  for i in df.columns[1:]]




train_plus_test = pd.merge(train_plus_test,df, left_on='State', right_on='State', how='left')
train_plus_test = train_plus_test [[
                'State','City','Type',
                'Population [2011]','Population [2011]__mean','Population [2011]__max','Population [2011]__min',
                'Sex Ratio' , 'Sex Ratio__mean','Sex Ratio__max','Sex Ratio__min', 
                'Median Age', 'Median Age__mean','Median Age__max','Median Age__min', 
                'Avg Temp','Avg Temp__mean', 'Avg Temp__max', 'Avg Temp__min',
                'SWM' , 'SWM__mean','SWM__max','SWM__min',
                'Toilets Avl', 'Toilets Avl__mean','Toilets Avl__max', 'Toilets Avl__min',
                'Water Purity', 'Water Purity__mean' ,'Water Purity__max', 'Water Purity__min',
                'H Index', 'H Index__mean','H Index__max', 'H Index__min',
                'Female Population','Female Population__mean','Female Population__max', 'Female Population__min',
                '# of hospitals','# of hospitals__mean','# of hospitals__max', '# of hospitals__min',
                'Foreign Visitors','Foreign Visitors__mean','Foreign Visitors__max','Foreign Visitors__min', 
                'Covid Cases','Covid Cases__mean','Covid Cases__max','Covid Cases__min'
             ]]




train = train_plus_test[~train_plus_test['Covid Cases'].isna()]
test1 = train_plus_test[ train_plus_test['Covid Cases'].isna()]
# train.shape, test1.shape






from catboost import Pool, CatBoostRegressor
            
features = ['# of hospitals',
 'Avg Temp',
 'Covid Cases__max',
 'Covid Cases__mean',
 'Covid Cases__min',
 'Foreign Visitors',
 'H Index',
 'H Index__max',
 'H Index__min',
 'Median Age',
 'Median Age__mean',
 'Population [2011]',
 'Population [2011]__max',
 'Population [2011]__mean',
 'Population [2011]__min',
 'SWM',
 'Sex Ratio',
 'Sex Ratio__mean',
 'Sex Ratio__min',
 'Toilets Avl',
 'Water Purity']



from sklearn.utils import shuffle

import numpy as np
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


from sklearn.metrics import mean_absolute_error

train  =  shuffle(train)

model = CatBoostRegressor(loss_function='MAE', verbose=False)
model.fit(Pool(train[features], train['Covid Cases']))
preds = model.predict(test1[features])

test1['Covid Cases']  =  preds
test1[['City', 'Covid Cases']].to_csv(path + 'Output file 01.csv', index=False, encoding='utf-8')

# In 02.ipynb, we forecast 'percentage_change' as +15.3% for the next month. <br> And, assuming every other column values (remain almost) the same, <br> we'll predict the 'Covid Cases' for 01-OCT using the above trained model.

In [ ]:
test1['Foreign Visitors'] = test1['Foreign Visitors']*(1+0.15338964585835002)
preds = model.predict(test1[features])

test1['Covid Cases']  =  preds
test1[['City', 'Covid Cases']].to_csv(path + 'Output file 02.csv', index=False, encoding='utf-8')